+ 논문 및 혁펜하임의 ATT강의를 참고하고 정리한 자료

# Swin Transformer

## Main Contribution

+ __Shifted window__기법을 사용해 high resolution에서 기존의 ViT가 가진 __높은 컴퓨팅 연산량을 해결__하였다.

크게보면, ViT에 Shifted windows, Patch merging, Relative position bias 세가지가 사용되었다.

## Shifted window based Self-Attention

<img src="https://drive.google.com/uc?id=1PkLlSEMiwBB7xpUegoBEqNUThftF0yxH" height=300>

+ 기존의 ViT는 이미지 전체의 모든 패치에 대해 MSA가 이뤄진다.
    + __Window based MSA 도입(W-MSA)__
+ 이렇게하면, global한 정보를 못본다
    + __Shifted window based MSA 도입(SW-MSA)__


자잘한 네모는 하나의 패치단위이고, 떨어져있는 네모가 윈도우이다. (위의 경우는 sindow size = (4, 4)
+ 그림을 보면 패치의 크기도 점점 커짐을 알 수 있는데 이를 __Patch Merging__기법이라 부른다.

### MSA와 W-MSA의 연산량 비교

+ $\Omega(\mathbf{MSA}) = 4hwC^2 + 2(hw)^2C$
+ $\Omega(\mathbf{W}$-$\mathbf{MSA}) = 4hwC^2 + 2M^2hwC$

윈도우 크기를 MxM이라하고, 패치단위 이미지 크기를 hxw, 차원 수를 C라고하자.

<img src="https://drive.google.com/uc?id=1vUzMjy8QYAyAhgda1AHy9QW9QO4g-3C1" height=250>

__Linear term__
+ 기존의 MSA는 QKV를 만들고 concat한 것을 nn.Linear하는 것이 각각 $(hw \times C)$ @ $(C \times C)$ 이므로 연산량은 $4hwC^2$  


__MSA term__
+ $\text{softmax} \bigg(\cfrac{\mathbf{QK^T}}{\sqrt{d_k}}\mathbf{V} \bigg)$에서, $QK^T$계산 : $(hw \times C)$ @ $(C \times hw)$
+ $V$랑 계산 : $(hw \times hw)$ @ $(hw \times C)$
+ 총 $2(hw)^2C$

__W-MSA term__
+ 여기서 윈도우 안에서만 어텐션해주니까 $2(M^2)^2C \times \cfrac{h}{M} \cdot \cfrac{w}{M} = 2M^2hwC$

즉, 계산복잡도가 이미지 크기에 선형적이다. (정확히는 패치단위에)  
M=7으로 고정해서 사용한다.

### Shifted Window 기법

<img src="https://drive.google.com/uc?id=1osH4b6TA_USTARpxd6AQmsf2kZbRj_7D" height=200>

위의 layer를 반복적으로 계속해서 적용해나가는 것.  
+ layer 1 : 4개의 Window
+ layer 1 + 1 : 9개의 Window

__non-overlapping window들을 이용해 정보를 교환하기 위함이다.__

<img src="https://drive.google.com/uc?id=1s5yDbHakuZ3hwlUI5LnrL_x84OlHdsmG" height=400>

위의 방법을 사용하려고보니, 두번째 레이어에서는 윈도우마다 크기가 다 다르고 심지어 더 많아진다.. (4 -> 9)  
__cyclic shift!__

<img src="https://drive.google.com/uc?id=1WAQRYfcCC23ha2LE7iZczWnyKzhfcXKS" height=220>

1. window를 크기에 맞춰 쪼개준다.
2. 남는 테두리 애들은 반대쪽으로(점대칭?) 옮겨준다.
3. 그러면 원래 window대로 4개가 있게 된다.
4. 각 window 내부에서 원래 자기 window에 해당하지 않는 patch들에는 masking을 해준다.
5. attention 해주고, 다시 원래의 window로 복원해준다.

### Relative position bias

$\text{Attention} = \text{softmax}\bigg( \cfrac{QK^T}{\sqrt{d}} + B \bigg)$

$B \in \mathbb{R}^{M^2 \times M^2}$이고, 각각의 헤드에 대해 따로 적용해준다.

학습가능한 파라미터이다.

<img src="https://drive.google.com/uc?id=1jUHRUgtJdy_9z50lrnuONwkoE01qdcL-" height=350>

1. attention하기 위해 주어진 이미지가 있다.(flatten하지 않은 상태)
2. 가로세로 크기가 M이라고하면, 한 패치를 기준으로 가장 먼 거리는 M-1이 될 것이다.
3. $\hat{B}$ : 좌우를 모두 고려하면 2M-1의 사이즈가 될 것이다. (두부게임..? 사람이 3명있고 일자로 서있다면 두부 5모까지만 가능)
4. 이제 원래의 이미지를 $\hat{B}$에 대조해보면서 각각의 번호에 대한 상대적인 위치값이 몇이 되는지를 B matrix에 기입해준다.
5. 이걸 attention matrix에 더해준다.

## Model architecture

<img src="https://drive.google.com/uc?id=1k71xZz5oPlWl-1uRid6SQ8rc5Pz3C4_A" height=300>

+ 각각의 Stage를 짝수의 개수로 해서 W-MSA, SW-MSA block을 번갈아 사용하도록 구성
+ Patch merging을 해서 총 Patch수는 계속해서 줄어든다. (4x4 -> 8x8 -> ..)
+ C 수는 그에 따라 늘려준다.
+ 왜 이렇게할까?
    + swin의 경우에는 간접적으로 정보를 전달했다면(타고, 타고 타면서..)
    + receptive field를 직접적으로 높이기 위함!
    

## 성능 비교

<img src="https://drive.google.com/uc?id=1RCv0ysyqretmwtmAY-Dq5mJxWCxgM3s_" width=450>

ImageNet분야에서 ViT의 성능 압도. 하지만, EffNet이랑 비교했을 때는 파라미터수랑 연산량 대비 엄청 큰차이는 없다. 다만, 출력량은 더 많다.